# Import modules

In [6]:
import os
import pandas as pd
from typing import List

# Constants

In [7]:
Train_path = "tmlcc-2021/mof_cif_train"

# Test load files

In [5]:
"""test = load_cif_file(f"{Train_path}/mof_unit_1.cif")"""

'test = load_cif_file(f"{Train_path}/mof_unit_1.cif")'

In [17]:
from utils.classes.adapters import CIF2PandasAdapter

cif2pandas_adapter = CIF2PandasAdapter()
results = cif2pandas_adapter.apply(f"{Train_path}/mof_unit_123.cif")


In [16]:
results["metadata"]

,0,1
0,_audit_creation_date,2012-10-26T19:46:07-0400
1,_symmetry_space_group_name_H-M,P1
2,_symmetry_Int_Tables_number,1
3,_space_group_crystal_system,triclinic
4,_cell_length_a,10.762679
5,_cell_length_b,9.62445
6,_cell_length_c,15.885493
7,_cell_angle_alpha,90.029202
8,_cell_angle_beta,92.771314
9,_cell_angle_gamma,90.020289


In [ ]:
metadata = pd.DataFrame(dataframes[0])
metadata

,0,1
0,_audit_creation_date,2013-03-05T16:41:16-0500
1,_symmetry_space_group_name_H-M,P1
2,_symmetry_Int_Tables_number,1
3,_space_group_crystal_system,triclinic
4,_cell_length_a,10.609882
5,_cell_length_b,10.643578
6,_cell_length_c,9.890832
7,_cell_angle_alpha,89.904234
8,_cell_angle_beta,91.242355
9,_cell_angle_gamma,90.261935


In [ ]:
loop_1 = pd.DataFrame(dataframes[1])
loop_1_fixed = loop_1[loop_1[1].notna()]
loop_1_fixed.columns = loop_1[loop_1[1].isna()][0]


0             _atom_site_label
1       _atom_site_type_symbol
2       _atom_site_description
3           _atom_site_fract_x
4           _atom_site_fract_y
5           _atom_site_fract_z
6    _atom_type_partial_charge
Name: 0, dtype: object

In [ ]:
loop_2 = pd.DataFrame(dataframes[2])
loop_2_fixed = loop_2[loop_2[1].notna()]
loop_2_fixed.columns = loop_2[loop_2[1].isna()][0]

In [ ]:
loop_2_fixed

,_geom_bond_atom_site_label_1,_geom_bond_atom_site_label_2,_geom_bond_distance,_ccdc_geom_bond_type
4,O4,Zn2,1.964878,S
5,Zn1,O7,1.964880,S
6,C10,C11,1.421566,A
7,C21,C22,1.527080,S
8,O19,H19,1.013968,S
...,...,...,...,...
83,O13,C25,1.411417,S
84,C21,H8,1.113555,S
85,C14,C15,1.497242,S
86,O6,C3,1.284366,A


In [ ]:

raw_dataframes.index[raw_dataframes[0] == "loop_"]

Int64Index([13, 97], dtype='int64')

In [ ]:
test.

[]

In [ ]:
test.get_all_unit_cell_sites()[0].f

<gemmi.SmallStructure.Site O1>

In [ ]:
ls -a

 ./                    'Getting started.ipynb'      .temp/
 ../                    init.ipynb                  tmlcc-2021/
 data.zip               notebook.ipynb              untitled-20210918-202702/
 .deepnote/             requirements.txt
 download_files.ipynb   SF_bikeshare_stations.csv


In [ ]:
from io import StringIO


gemmi.read_small_structure(StringIO('\n'.join(lines)))

TypeError: a bytes-like object is required, not 'str'

In [ ]:

cif_object = gemmi.read_small_structure("mof_cif_train/mof_cif_train/mof_unit_1.cif")

ValueError: mof_cif_train/mof_cif_train/mof_unit_1.cif:3:45: parse error

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf8541de-dbc3-45f6-bc1e-4fa446cacbcd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>